In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.edge.service import Service
from itertools import combinations

import time


In [ ]:
EDGE_DRIVER_PATH = "edgedriver_win32/msedgedriver.exe"
driver = webdriver.Edge(service=Service(EDGE_DRIVER_PATH))

In [ ]:
wait = WebDriverWait(driver, 10)

In [ ]:
driver.get("https://rustypot.com/coinflip")

In [ ]:
def get_possible_bet_values(inventory_items_values, max_items=3):
    possible_bet_values = {}
    for r in range(1, max_items):
        for combo in combinations(inventory_items_values, r):
            s = sum(combo)
            max_tax = s * 2 * 0.05
            good_combo = True
            for item_value in combo:
                if item_value < max_tax:
                    good_combo = False
            if good_combo:
                possible_bet_values[s] = combo
    possible_bet_values_sorted = dict(sorted(possible_bet_values.items()))
    
    return possible_bet_values_sorted

In [ ]:
def get_inventory_items():
    create_a_game_buttom_locator = (By.CLASS_NAME, "createCoinflipButton")
    create_a_game_buttom = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located(create_a_game_buttom_locator)
    )
    create_a_game_buttom.click()
    inventory_items_div_locator = (By.CLASS_NAME, "depositInventory")
    inventory_items_div = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located(inventory_items_div_locator)
    )
    time.sleep(1)
    inventory_items = inventory_items_div.find_elements(By.CLASS_NAME, "InventoryItem")
    inventory_items_values = []
    for inventory_item in inventory_items:
        inventory_items_values.append(float(inventory_item.text[1:]))
    
    driver.get("https://rustypot.com/coinflip")
    
    print("inventory items gathered: ", inventory_items_values)
    
    return inventory_items_values

In [ ]:
inventory_items_values = [62.31, 31.75, 31.75, 20.86, 17.38, 13.34, 13.34]

# inventory_items = [i for i in range(1, 21, 3)]
possible_bet_values = get_possible_bet_values(inventory_items_values)
possible_bet_values

In [ ]:
try:
    inventory_items_values = get_inventory_items()
except Exception as e:
    driver.get("https://rustypot.com/coinflip")
    time.sleep(2)
    inventory_items_values = get_inventory_items()
finally:
    if len(inventory_items_values) == 0:
        get_inventory_items()
        
possible_bet_values = get_possible_bet_values(inventory_items_values)
print(possible_bet_values)

In [ ]:
bets = []
counter = 0
while True:

    active_coinflips = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "ActiveCoinflips")))
    coinflip_row_divs = active_coinflips.find_elements(By.CLASS_NAME, "coinflip")
    
    # print(f"{len(coinflip_divs)} active games")
    for coinflip_div in coinflip_row_divs:
        coinflip_value = float(coinflip_div.get_attribute("coinflip-value"))
        coinflip_items = coinflip_div.find_element(By.CLASS_NAME, "coinflipGameItems")
        
        if "items" in coinflip_items.text:
            # print("more than 5 items...")
            continue
        img_elements = coinflip_items.find_elements(By.TAG_NAME, "img")
        
        coinflip_items_values = []
        for img in img_elements:
            title = img.get_attribute("data-original-title")
            if "|" in title:
                price_str = title.split("$")[-1]
                price = float(price_str)
                coinflip_items_values.append(price)
    
        possible_tax_value = coinflip_value * 2 * 0.05
        cheapest_item_value = coinflip_items_values[-1]
        
        if cheapest_item_value < possible_tax_value:
            # print(f"not profitable")
            continue
        
        # to join a cf, i need to deposit minimum value
        for bet_value in possible_bet_values:
            if  coinflip_value * 0.9 < bet_value < coinflip_value * 0.92 and coinflip_value not in bets:
                bets.append(coinflip_value)

                print(f"{coinflip_value=} -> "
                      f"{coinflip_items_values=} -> "
                      f"Bet items= {possible_bet_values[bet_value]}")
                break
        # print(f"{coinflip_value=} -> {possible_tax_value=:.2f} -> {cheapest_item_value=:.2f}")
    counter += 1
    # if counter % 100 == 0:
    #     print("active - ", counter)
    time.sleep(1)